In [25]:
using JuMP, Gurobi, LightGraphs
model = direct_model(Gurobi.Optimizer())
global arcs = [1 2; 2 3; 3 5; 1 4; 4 5]
global c = [2.0 4 3 5 5;
            1.0 5 3 5 5;
            2.0 3 3 5 5]
global p = [0.25,0.5,0.25]
global alpha = 0.6
global numArcs = length(arcs[:, 1])
global numNodes = 5
global delta = 1.0
global S = 3
global M = 100

@variable(model, d[1:numArcs] >=0)
@variable(model, d_[1:numArcs] >=0)
@variable(model, y[1:S, 1:numNodes] >= 0)#1e6)
@variable(model, z[1:S], Bin)

@objective(model, Min, sum(d[a] + d_[a] for a = 1:numArcs))

@constraint(model, sum(p[s]*z[s] for s =1:S) >= alpha)
# println(c[2,3])
for s = 1:S
    global c, arcs, numNodes, delta, M
    @constraint(model, y[s,1] - y[s,4] == c[s,4] + d[4] - d_[4])
    @constraint(model, y[s,4] - y[s,5] == c[s,5] + d[5] - d_[5])
    @constraint(model, y[s,1] - y[s,2] <= c[s,1] + d[1] - d_[1] - delta + M*(1-z[s]))
    @constraint(model, y[s,2] - y[s,3] <= c[s,2] + d[2] - d_[2] + M*(1-z[s]))
    @constraint(model, y[s,3] - y[s,5] <= c[s,3] + d[3] - d_[3] + M*(1-z[s]))
end
println(model)
optimize!(model)
sol_d = JuMP.value.(d)
sol_z = JuMP.value.(z)
sol_d_= JuMP.value.(d_)
obj = JuMP.objective_value.(model)

println("Obj = ", obj)
println("sol_d = ",sol_d)
println("sol_d_ = ", sol_d_)
println("z = ", sol_z)


Academic license - for non-commercial use only - expires 2022-12-04
Min d[1] + d[2] + d[3] + d[4] + d[5] + d_[1] + d_[2] + d_[3] + d_[4] + d_[5]
Subject to
 z[1] binary
 z[2] binary
 z[3] binary
 -d[4] + d_[4] + y[1,1] - y[1,4] == 5.0
 -d[5] + d_[5] + y[1,4] - y[1,5] == 5.0
 -d[4] + d_[4] + y[2,1] - y[2,4] == 5.0
 -d[5] + d_[5] + y[2,4] - y[2,5] == 5.0
 -d[4] + d_[4] + y[3,1] - y[3,4] == 5.0
 -d[5] + d_[5] + y[3,4] - y[3,5] == 5.0
 d[1] >= 0.0
 d[2] >= 0.0
 d[3] >= 0.0
 d[4] >= 0.0
 d[5] >= 0.0
 d_[1] >= 0.0
 d_[2] >= 0.0
 d_[3] >= 0.0
 d_[4] >= 0.0
 d_[5] >= 0.0
 y[1,1] >= 0.0
 y[2,1] >= 0.0
 y[3,1] >= 0.0
 y[1,2] >= 0.0
 y[2,2] >= 0.0
 y[3,2] >= 0.0
 y[1,3] >= 0.0
 y[2,3] >= 0.0
 y[3,3] >= 0.0
 y[1,4] >= 0.0
 y[2,4] >= 0.0
 y[3,4] >= 0.0
 y[1,5] >= 0.0
 y[2,5] >= 0.0
 y[3,5] >= 0.0
 -d[1] + d_[1] + y[1,1] - y[1,2] + 100 z[1] <= 101.0
 -d[2] + d_[2] + y[1,2] - y[1,3] + 100 z[1] <= 104.0
 -d[3] + d_[3] + y[1,3] - y[1,5] + 100 z[1] <= 103.0
 -d[1] + d_[1] + y[2,1] - y[2,2] + 100 z[2] <=